In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%pwd

In [ ]:
%%bash
cd models/research
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

In [ ]:
%pwd

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
train_csv = pd.read_csv('../input/global-wheat-detection/train.csv')

In [ ]:
bboxes = np.stack(train_csv['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['xmin', 'ymin', 'width', 'height']):
    train_csv[column] = bboxes[:,i]
    
train_csv["xmax"] = train_csv.apply(lambda col: col.xmin + col.width, axis=1)
train_csv["ymax"] = train_csv.apply(lambda col: col.ymin + col.height, axis = 1)
train_csv.drop(columns=['bbox'], inplace=True)

In [ ]:
xmax = np.array(train_csv["xmax"].values.tolist())
ymax = np.array(train_csv["ymax"].values.tolist())
train_csv["xmax"] = np.where(xmax > 1024, 1024, xmax).tolist()
train_csv["ymax"] = np.where(ymax > 1024, 1024, ymax).tolist()

In [ ]:
train_csv["class"] = "wheat"

In [ ]:
del train_csv["source"]

In [ ]:
train_csv['filename']=train_csv['image_id']
del train_csv['image_id']

In [ ]:
train_csv['filename'] = train_csv['filename'] + '.jpg'

In [ ]:
train_csv.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val = train_test_split(train_csv, test_size = 0.2)

In [ ]:
x_train.to_csv('training.csv',index=False)
x_val.to_csv('eval.csv',index=False)

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

In [ ]:
!tar --gunzip --extract --verbose --file=faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

In [ ]:
open("generate_tfrecord.py","w+")

In [ ]:
%pwd

In [ ]:
%%writefile '/kaggle/working/generate_tfrecord.py'
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.compat.v1.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('image_dir', '', 'Path to images')
FLAGS = flags.FLAGS


# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'wheat':
        return 1
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.compat.v1.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(FLAGS.image_dir)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.compat.v1.app.run()

In [ ]:
open("label_map.txt","w+")

In [ ]:
%%writefile '/kaggle/working/label_map.txt'
item {
  id: 1
  name: 'wheat'
}

In [ ]:
!python /kaggle/working/generate_tfrecord.py --csv_input=/kaggle/working/training.csv  --output_path=train.record --image_dir=../input/global-wheat-detection/train

In [ ]:
!python3 /kaggle/working/generate_tfrecord.py --csv_input=eval.csv  --output_path=eval.record --image_dir=../input/global-wheat-detection/train

In [ ]:
open("pipeline.config","w+")

In [ ]:
%%writefile '/kaggle/working/models/research/object_detection/configs/tf2/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config'
model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 640
        max_dimension: 640
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet50_keras'
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
        share_box_across_classes: true
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
    use_static_shapes: true
    use_matmul_crop_and_resize: true
    clip_anchors_to_image: true
    use_static_balanced_label_sampler: true
    use_matmul_gather_in_matcher: true
  }
}

train_config: {
  batch_size: 5
  sync_replicas: true
  startup_delay_steps: 0
  replicas_to_aggregate: 8
  num_steps: 25000
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        cosine_decay_learning_rate {
          learning_rate_base: .04
          total_steps: 25000
          warmup_learning_rate: .013333
          warmup_steps: 2000
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "/kaggle/working/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "detection"
  data_augmentation_options {
    random_horizontal_flip {
    }
  }

  max_number_of_boxes: 150
  unpad_groundtruth_tensors: false
  use_bfloat16: true  # works only on TPUs
}

train_input_reader: {
  label_map_path: "/kaggle/working/label_map.txt"
  tf_record_input_reader {
    input_path: "/kaggle/working/train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/kaggle/working/label_map.txt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/kaggle/working/eval.record"
  }
}

In [ ]:
!mkdir ./tuned

In [ ]:
%pwd

In [ ]:
%cd /kaggle/working/models/research/

In [ ]:
!python object_detection/model_main_tf2.py \
    --num_train_steps=2500 \
    --sample_1_of_n_eval_examples=1 \
    --pipeline_config_path=/kaggle/working/models/research/object_detection/configs/tf2/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config \
    --model_dir=/kaggle/working/tuned \
    --alsologtostderr

In [ ]:
%pwd


In [ ]:
!python object_detection/exporter_main_v2.py \
    --pipeline_config_path /kaggle/working/models/research/object_detection/configs/tf2/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config \
    --trained_checkpoint_dir /kaggle/working/tuned \
    --output_directory /kaggle/working/output

In [ ]:
%cd /kaggle/working

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile

In [ ]:
def load_model():
    model = tf.saved_model.load('/kaggle/working/output/saved_model')
    return model

In [ ]:
PATH_TO_LABELS = '/kaggle/working/label_map.txt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('../input/global-wheat-detection/test')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [ ]:
detection_model = load_model()

In [ ]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]

    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    if 'detection_masks' in output_dict:
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
        
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
        
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return output_dict

In [ ]:
%pwd


In [ ]:
import csv
filename = 'submission1.csv'
fields = ['image_id', 'PredictionString']
with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    
    for image_path in TEST_IMAGE_PATHS:
        image_np = np.array(Image.open(image_path))
        hel = run_inference_for_single_image(detection_model,image_np)
        h = round(pd.DataFrame(hel['detection_boxes'])*1024,0)
        r = pd.DataFrame(hel['detection_scores'])
        h['D'] = r
        score = h['D']
        xmin = h[1]
        xmax = h[3]
        ymin = h[0]
        ymax = h[2]
        for i in np.arange(0,len(xmin)):
            if(score[i]>0.5):
                s = ["{:.1f}".format(score[i]),int(xmin[i]), int(ymin[i]), int(xmax[i]-xmin[i]), int(ymax[i]-ymin[i])]
                x = ' '.join([str(elem) for elem in s])
                y = str(image_path).strip('.jpg') 
                row = [y, x]
                csvwriter.writerow(row)



In [ ]:
import pandas as pd
data = pd.read_csv('/kaggle/working/submission1.csv')

In [ ]:
import numpy as np 
for i in np.arange(0,len(data['image_id'])):
    data['image_id'][i] = data['image_id'][i][35:44] 

In [ ]:
data.to_csv('submission.csv',index = False)